In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('raw_data/2021 (A).csv', header=[0,1])

/home/johnfred2k/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (11,12,17,22,27,32,37,42,47,52,57,62,67,72,77,82,87,92,97,102,107,112,117,122,127,132,137,142,147,152,157,162,167,172,177,182,187,192,197,202,207,212,217,222,227,232,237,242,247,252,257,262,267,272,277,282,287,292,297,302,307,312,317,322,327,332,337,342,347,352,357,362,367,372,377,382,387,392,397,402,407,412,417,422,427,432,437,442,447,452,457,462,467,472,477,482,487,492,497,502,507,512,517,522,527,532,537,542,547,552,557,562,567,572,577,582,587,592,597,602,607,612,617,622,627,632,637,642,647,652,657,662,667,672,677,682,687,692,697,702,707,712,717,722,727,732,737,742,747,752,757,762,767,772,777,782,787,792,797,802,807,812,817,822,827,832,837,842,847,852,857,862,867,872,877,882,887,892,897,902,907,912,917,922,927,932,937,942,947,952,957,962,967,972,977,982,987,992,997,1002,1007,1012,1017,1022,1027,1032,1037,1042,1047,1052,1057,1062,1067,1072,1077,

In [4]:
df.shape

(55187, 1839)

In [5]:
df = df[df['Check']['Check'] > 0]
df = df.drop(columns='Check')
df.shape

/home/johnfred2k/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


(45161, 1838)

In [6]:
first_dict = {}

for i in df[[('Unnamed: 0_level_0', 'id')]].values[:15000]:
    first_dict[i[0]]=df[df[('Unnamed: 0_level_0', 'id')] == i[0]]
    first_dict[i[0]].drop(columns=['Unnamed: 1_level_0', 'Unnamed: 2_level_0', 'Unnamed: 3_level_0', 'Unnamed: 4_level_0', 'Unnamed: 5_level_0', 'Unnamed: 6_level_0', 'Unnamed: 7_level_0', 'Unnamed: 8_level_0', 'Unnamed: 9_level_0', 'Unnamed: 10_level_0', 'Unnamed: 11_level_0', 'Unnamed: 12_level_0'], inplace=True)
    first_dict[i[0]].drop(columns=[('Unnamed: 0_level_0', 'id')], inplace=True)

/home/johnfred2k/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
second_dict = {}

for k, v in first_dict.items():
    second_dict[k] = v.unstack().reset_index().drop(columns='level_2').rename(columns={'level_0':'date', 'level_1': 'type', 0:'value'})
    second_dict[k]['value']=second_dict[k]['value'].apply(lambda x: float(str(x).replace(',', '.')))

ValueError: could not convert string to float: '1 500'

In [ ]:
third_dict = {}

for k, v in second_dict.items():
    third_dict[k] = pd.pivot_table(data=v, index='date', columns='type', values='value', aggfunc=np.mean, fill_value=0).reset_index()
    third_dict[k]['id'] = k

In [ ]:
clean_df = pd.concat(third_dict.values())

In [ ]:
df_test_d2 = df.iloc[: ,:13]
df_test_d1 = pd.DataFrame([])
for col in df_test_d2:
    df_test_d1[col[0]] = df_test_d2[col[0]][col[1]]

df_test_d1 = df_test_d1.rename(columns={'Unnamed: 0_level_0': 'id', 'Unnamed: 1_level_0':'Store ID', 'Unnamed: 2_level_0':'Item ID', 'Unnamed: 3_level_0':'Store Category', 'Unnamed: 4_level_0':'Store Segment', 'Unnamed: 5_level_0':'Store Region', 'Unnamed: 6_level_0':'City', 'Unnamed: 7_level_0': 'Store Address', 'Unnamed: 8_level_0': 'Item First Supplying Date', 'Unnamed: 9_level_0': 'Item Last Supplying Date', 'Unnamed: 10_level_0': 'Item Price', 'Unnamed: 11_level_0': 'Pickup Time Start Time of Day', 'Unnamed: 12_level_0': 'Pickup Time End Time of Day'})
df_test_d1.head(2)         

In [ ]:
final_df = pd.merge(df_test_d1, clean_df, on=['id'])
final_df.head()

In [ ]:
final_df.to_csv('2021_processed_1.csv')